In [1]:
import modin.pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/all_sources_metadata_2020-03-13.csv')

In [4]:
df.head(3)

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
0,c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2.002765e+09,#3252,True
1,53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3.003431e+09,#1861,True
2,210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3.006065e+09,#1043,True


In [5]:
df.info()

To request implementation, send an email to feature_requests@modin.org.


<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 29500 entries, 0 to 29499
Data columns (total 14 columns):
sha                            17420 non-null object
source_x                       29500 non-null object
title                          29130 non-null object
doi                            26357 non-null object
pmcid                          27337 non-null object
pubmed_id                      16730 non-null float64
license                        17692 non-null object
abstract                       26553 non-null object
publish_time                   18248 non-null object
authors                        28554 non-null object
journal                        17791 non-null object
Microsoft Academic Paper ID    1134 non-null float64
WHO #Covidence                 1236 non-null object
has_full_text                  17420 non-null object
dtypes: float64(2), object(12)
memory usage: 3.2+ MB


In [6]:
df.has_full_text.value_counts()

True     13219
False     4201
dtype: int64

In [7]:
df.abstract.isnull().sum()

2947

In [8]:
df.shape

(29500, 14)

In [10]:
len(df.publish_time.value_counts())

4499

In [11]:
df.source_x.value_counts()

PMC        27337
CZI         1236
biorxiv      566
medrxiv      361
dtype: int64